In [1]:
from itertools import izip
import pickle
import pandas as pd
import numpy as np
import graph_tool as gt
import graph_tool.topology as top
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
import matplotlib.pylab as plt
%matplotlib inline 
from operator import itemgetter
import warnings
warnings.filterwarnings('ignore')




In [2]:
filename = 'ucidata.txt'
g = gt.load_graph_from_csv("./data/{}".format(filename), directed=True, eprop_types=['int'], eprop_names=['weight'], string_vals=True)
g2 = gt.load_graph_from_csv("./data/{}".format(filename), directed=False, eprop_types=['int'], eprop_names=['weight'], string_vals=True)
print(g.is_directed())
print(g2.is_directed())


True
False


In [3]:

for v in g.vertices():
    print("process node:"+repr(int(v)))
    for e in v.out_edges():
        print(e)
    for w in v.out_neighbours():
       print(w)

process node:0
(0, 1)
1
process node:1
process node:2
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 13)
(2, 16)
(2, 17)
3
4
5
6
7
13
16
17
process node:3
(3, 4)
(3, 6)
(3, 7)
(3, 10)
(3, 11)
(3, 16)
(3, 17)
4
6
7
10
11
16
17
process node:4
(4, 5)
(4, 7)
(4, 8)
(4, 9)
5
7
8
9
process node:5
(5, 9)
9
process node:6
(6, 8)
(6, 10)
(6, 15)
(6, 16)
(6, 17)
8
10
15
16
17
process node:7
(7, 8)
(7, 9)
(7, 10)
(7, 12)
(7, 13)
(7, 14)
(7, 17)
8
9
10
12
13
14
17
process node:8
(8, 9)
(8, 12)
(8, 13)
(8, 14)
9
12
13
14
process node:9
(9, 12)
(9, 13)
(9, 15)
12
13
15
process node:10
(10, 11)
(10, 12)
(10, 14)
(10, 16)
11
12
14
16
process node:11
(11, 12)
(11, 14)
(11, 16)
12
14
16
process node:12
(12, 13)
(12, 14)
(12, 16)
(12, 17)
13
14
16
17
process node:13
(13, 15)
(13, 16)
(13, 17)
15
16
17
process node:14
(14, 15)
(14, 16)
(14, 17)
15
16
17
process node:15
(15, 17)
17
process node:16
(16, 17)
17
process node:17


In [4]:
from collections import Counter

def deg(node, sign='negative', mode='out'):
    '''
    @params:
        node (graph_tool node object): node id
        sign (string): 'negative' or 'positive'  
        mode (string): 'in', 'out', or 'all' 
    @returns
        degree (int): corresponding degree 
    '''
    if sign == 'negative':
        if mode == 'out' :
            return Counter([g.edge_properties['weight'][e] for e in node.out_edges()])[-1]
        if mode == 'in' :
            return Counter([g.edge_properties['weight'][e] for e in node.in_edges()])[-1]
        
    elif sign == 'positive':
        if mode == 'out' :
            return Counter([g.edge_properties['weight'][e] for e in node.out_edges()])[1]
        if mode == 'in' :
            return Counter([g.edge_properties['weight'][e] for e in node.in_edges()])[1]
        
def degree_features(u, v):
    d1 = deg(v, 'positive', 'in')
    d2 = deg(v, 'negative', 'in')
    d3 = deg(u, 'positive', 'out')
    d4 = deg(u, 'negative', 'out')
    return [ d1,\
             d2,\
             d3,\
             d4,\
             d1 + d2,\
             d3 + d4,
           ]

In [5]:
def shingle_path(path):
    return set([tuple(path[i:i + 2]) for i in range(len(path)-1)])

def distinct_path(g, s, t, c=3):
    g.set_directed(False)
    if s == t:
        return ((s,t), [], 0)
    paths = list(top.all_paths(g, s, t,cutoff=c))
    sorted_paths = sorted(paths, key=lambda x: len(x))
    seen_edges = set()
    good_paths = []
    for path in sorted_paths:
        edges = shingle_path(path)
        if len(edges) < c:
            continue
        if not(edges.intersection(seen_edges)):
            good_paths.append(edges)
            seen_edges = seen_edges.union(edges)
    return((s,t), good_paths, len(good_paths))


def distinct_path2(g, s, t, c=3):
    if s == t:
        return ((s,t), [], 0)
    paths = list(top.all_paths(g, s, t,cutoff=c))
    sorted_paths = sorted(paths, key=lambda x: len(x))
    seen_edges = set()
    signs = [0, 0] #index[0] == positve, index[1] == negative path
    for path in sorted_paths:
        edges = shingle_path(path)
        if len(edges) < c:
            continue
        if not(edges.intersection(seen_edges)):
            seen_edges = seen_edges.union(edges)
            sign = 1
            for edge in edges:
                sign *= g.edge_properties['weight'][edge]
            if sign == 1:
                signs[1] += 1
            elif sign == -1:
                signs[0] += 1
    return((s,t), signs)



In [6]:
def path_of_length_2(filename):
    path_of_2 = dict()
    counter = 0
    g.set_directed(False)
    for edge in list(g.edges()):
        s, t = edge.source().__hash__(), edge.target().__hash__()
        (n, n2), paths, num = distinct_path(g, s, t, 2)
        if num > 0:
            path_of_2[(s, t)] = paths
        counter += 1
    pickle.dump(path_of_2, open("./data/{}.paths_of_length_2.pk".format(filename), 'wb'),protocol=2 )
    
    print("finished path of length 2 for {}".format(filename))
    g.set_directed(True)

def path_of_length_3(filename):
    path_of_3 = dict()
    counter = 0
    g.set_directed(False)
    for edge in list(g.edges()):
        s, t = edge.source().__hash__(), edge.target().__hash__()
        (n, n2), paths, num = distinct_path(g, s, t, 3)
        if num > 0:
            #TODO: make sure this __hash__ is really what you think it is!!
            path_of_3[(s, t)] = paths
        counter += 1
    pickle.dump(path_of_3, open("./data/{}.paths_of_length_3.pk".format(filename), 'wb'), protocol=2)
    counts_3 = dict()
    for key in path_of_3:
        signs = [0, 0] #index[0] == positve, index[1] == negative path
        for path in path_of_3[key]:
            sign = 1
            for edge in path:
                sign *= g.edge_properties['weight'][edge]
            if sign == 1:
                signs[1] += 1
            elif sign == -1:
                signs[0] += 1
        counts_3[key] = signs
    pickle.dump(counts_3, open("./data/{}.paths_of_length_3_counts.pk".format(filename), 'wb'),protocol=2 )
    print("finished path of length 3 for {}".format(filename))
    g.set_directed(True)

def path_of_length_4(filename):
    path_of_4 = dict()
    counter = 0
    g.set_directed(False)
    for edge in list(g.edges()):
        s, t = edge.source().__hash__(), edge.target().__hash__()
        (n, n2), paths, num = distinct_path(g, s, t, 4)
        if num > 0:
            path_of_4[(s, t)] = paths
        counter += 1
    pickle.dump(path_of_4, open("./data/{}.paths_of_length_4.pk".format(filename), 'wb'), protocol=2)
    
    counts_4 = dict()
    for key in path_of_4:
        signs = [0, 0] #index[0] == positve, index[1] == negative path
        for path in path_of_4[key]:
            sign = 1
            for edge in path:
                sign *= g.edge_properties['weight'][edge]
            if sign == 1:
                signs[1] += 1
            elif sign == -1:
                signs[0] += 1
        counts_4[key] = signs
    pickle.dump(counts_4, open("./data/{}.paths_of_length_4_counts.pk".format(filename), 'wb'),protocol=2 )
    print("finished path of length 4 for {}".format(filename))
    g.set_directed(True)
    
    
def get_paths(filename):
    path_of_length_2(filename)
    path_of_length_3(filename)
    path_of_length_4(filename)

In [7]:
get_paths(filename)

finished path of length 2 for ucidata.txt
finished path of length 3 for ucidata.txt
finished path of length 4 for ucidata.txt


In [8]:
triples = pickle.load(open("./data/{}.paths_of_length_2.pk".format(filename), 'rb'))

In [9]:
triples

{(2, 3): [{(2, 4), (4, 3)},
  {(2, 6), (6, 3)},
  {(2, 7), (7, 3)},
  {(2, 16), (16, 3)},
  {(2, 17), (17, 3)}],
 (2, 4): [{(2, 3), (3, 4)}, {(2, 5), (5, 4)}, {(2, 7), (7, 4)}],
 (2, 5): [{(2, 4), (4, 5)}],
 (2, 6): [{(2, 3), (3, 6)}, {(2, 16), (16, 6)}, {(2, 17), (17, 6)}],
 (2, 7): [{(2, 3), (3, 7)},
  {(2, 4), (4, 7)},
  {(2, 13), (13, 7)},
  {(2, 17), (17, 7)}],
 (2, 13): [{(2, 7), (7, 13)}, {(2, 16), (16, 13)}, {(2, 17), (17, 13)}],
 (2, 16): [{(2, 3), (3, 16)},
  {(2, 6), (6, 16)},
  {(2, 13), (13, 16)},
  {(2, 17), (17, 16)}],
 (2, 17): [{(2, 3), (3, 17)},
  {(2, 6), (6, 17)},
  {(2, 7), (7, 17)},
  {(2, 13), (13, 17)},
  {(2, 16), (16, 17)}],
 (3, 4): [{(3, 7), (7, 4)}, {(2, 4), (3, 2)}],
 (3, 6): [{(3, 10), (10, 6)},
  {(3, 16), (16, 6)},
  {(3, 17), (17, 6)},
  {(2, 6), (3, 2)}],
 (3, 7): [{(3, 4), (4, 7)},
  {(3, 10), (10, 7)},
  {(3, 17), (17, 7)},
  {(2, 7), (3, 2)}],
 (3, 10): [{(3, 6), (6, 10)},
  {(3, 7), (7, 10)},
  {(3, 11), (11, 10)},
  {(3, 16), (16, 10)}],
 (3, 11)

In [10]:
edges2triad = dict()
for (pair, triad) in triples.items():
    keep = []
    for ((a, b), (c, d)) in triad:
        keep.extend(list(filter(lambda x: not(x in pair), [a,b,c,d])))
    edges2triad[pair] = keep
    
from collections import defaultdict
edge2weight_defaultzero = defaultdict(int)
for edge in g.edges():
    edge2weight_defaultzero[(edge.source().__hash__(), edge.target().__hash__())]  = g.edge_properties['weight'][edge]

In [11]:
def triads(s, t):
    ''' counts the number of triads for each type specified in the paper
    @params:
    s (int): index of source node 
    t (int): index of target node 
    @returns:
    number_of_triads (list): this 16-element list of ints denotes the number 
                             of each type  triad

    ''' 
    number_of_triads = [0]*16 
    try:
        nodes_to_check = set( edges2triad[(s,t)])
    except KeyError:
        #nodes_to_check =  edges2triad[(t,s)]
        return  number_of_triads
    g.set_directed(True)
    for u in nodes_to_check:
        #print "Testing triangle: "+ repr( int(s) ) +"," + repr( int(t) ) +"," + repr( int(u) ) +"," 
        ut = edge2weight_defaultzero[(u, t)]    
        tu = edge2weight_defaultzero[(t, u)]
        us = edge2weight_defaultzero[(u, s)]
        su = edge2weight_defaultzero[(s, u)]
        #print("Neighbor :"+repr(u)+" ut="+repr(ut)+" tu="+repr(tu)+" us="+repr(us)+" su="+repr(su))
        offset =0
        if  su == 1 and ut == 1: 
                number_of_triads[offset] += 1
        if  su == 1 and ut == -1: 
                number_of_triads[offset+1] += 1
        if  su == -1 and ut == 1: 
                number_of_triads[offset+2] += 1
        if  su == -1 and ut == -1: 
                number_of_triads[offset+3] += 1
                
        if  us == 1 and ut == 1: 
                number_of_triads[offset+4] += 1
        if  us == -1 and ut == 1: 
                number_of_triads[offset+5] += 1
        if  us == +1 and ut == -1: 
                number_of_triads[offset+6] += 1
        if  us == -1 and ut == -1: 
                number_of_triads[offset+7] += 1
          
        if  su == 1 and tu == 1: 
                number_of_triads[offset+8] += 1
        if  su == 1 and tu == -1: 
                number_of_triads[offset+9] += 1
        if  su == -1 and tu == 1: 
                number_of_triads[offset+10] += 1
        if  su == -1 and tu == -1: 
                number_of_triads[offset+11] += 1
        
        if  us == 1 and tu == 1: 
                number_of_triads[offset+12] += 1
        if  us == 1 and tu == -1: 
                number_of_triads[offset+13] += 1
        if  us == -1 and tu == 1: 
                number_of_triads[offset+14] += 1
        if  us == -1 and tu == -1: 
                number_of_triads[offset+15] += 1
        print(number_of_triads)  
        
    return number_of_triads

In [12]:
all_graph_features = dict()
counter = 0
for edge in list(g.edges()):
    s, t = edge.source().__hash__(), edge.target().__hash__()
    #print(repr(s)+"\t"+repr(t))
    all_graph_features[(s, t)] = degree_features(edge.source(), edge.target())
    counter += 1
print(counter)

59


In [13]:
pickle.dump(all_graph_features, open("./data/{}.graph_features.pk".format(filename), 'wb'),protocol=2)

In [14]:
all_triad_features= dict()
counter = 0
log = []
for edge in list(g.edges()):
    s, t = edge.source().__hash__(), edge.target().__hash__()
    #print("Processing edge: "+repr(s)+","+repr(t))
    try:
        all_triad_features[(s,t)] = triads(s, t)
        counter += 1
    except KeyError:
        all_triad_features[(s,t)] = [0]*16
        log.append((s, t))
print(counter)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0

In [15]:
pickle.dump(all_triad_features, open("./data/{}.triad_features.pk".format(filename), 'wb'),protocol=2)
cols = list(zip(*all_triad_features.values()))

In [16]:
all_triad_features

{(0, 1): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 (2, 3): [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0, 0, 0, 0],
 (2, 4): [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
 (2, 5): [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 (2, 6): [0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
 (2, 7): [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
 (2, 13): [0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
 (2, 16): [1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 (2, 17): [2, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 (3, 4): [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 (3, 6): [0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0],
 (3, 7): [0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 (3, 10): [0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
 (3, 11): [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 (3, 16): [0, 0, 0, 3, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 (3, 17): [1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 (4, 5): [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 

In [17]:
from collections import Counter
Counter([len(x) for x in all_triad_features.values()])

Counter({16: 59})

In [18]:
g2 = gt.load_graph_from_csv("./data/{}".format(filename), directed=False, eprop_types=['int'], eprop_names=['weight'], string_vals=True)
common_neighbors = {}
for edge in list(g2.edges()):
    u, v = edge.source(), edge.target()
    neighbors = set(u.all_neighbours()) & set(v.all_neighbours())
    common_neighbors[(u.__hash__(), v.__hash__())] = len(neighbors)
    
pickle.dump(common_neighbors, open("./data/{}.common_neighbors.pk".format(filename), 'wb'),protocol=2)

In [19]:
common_neighbors

{(0, 1): 0,
 (2, 3): 5,
 (2, 4): 3,
 (2, 5): 1,
 (2, 6): 3,
 (2, 7): 4,
 (2, 13): 3,
 (2, 16): 4,
 (2, 17): 5,
 (3, 4): 2,
 (3, 6): 4,
 (3, 7): 4,
 (3, 10): 4,
 (3, 11): 2,
 (3, 16): 5,
 (3, 17): 4,
 (4, 5): 2,
 (4, 7): 4,
 (4, 8): 2,
 (4, 9): 3,
 (5, 9): 1,
 (6, 8): 0,
 (6, 10): 2,
 (6, 15): 1,
 (6, 16): 4,
 (6, 17): 4,
 (7, 8): 5,
 (7, 9): 4,
 (7, 10): 3,
 (7, 12): 6,
 (7, 13): 5,
 (7, 14): 4,
 (7, 17): 5,
 (8, 9): 4,
 (8, 12): 4,
 (8, 13): 3,
 (8, 14): 2,
 (9, 12): 3,
 (9, 13): 4,
 (9, 15): 1,
 (10, 11): 4,
 (10, 12): 4,
 (10, 14): 4,
 (10, 16): 5,
 (11, 12): 3,
 (11, 14): 3,
 (11, 16): 4,
 (12, 13): 5,
 (12, 14): 6,
 (12, 16): 5,
 (12, 17): 4,
 (13, 15): 2,
 (13, 16): 3,
 (13, 17): 5,
 (14, 15): 1,
 (14, 16): 4,
 (14, 17): 4,
 (15, 17): 3,
 (16, 17): 6}